## Multinomial and Bernoulli Naive Bayes

For understanding Multinomial and Bernoulli Naive Bayes, we will start with a small example and understand the end to end process. In another notebook, we will build a full-fledged email spam classifier.

To start with, let's take a few sentences and classify them in two different classes - *education* or *cinema*. Each sentence will represent one document. In real-world cases, a document be any piece of text such as an email, a news article, a book review, a tweet etc.
The analysis and the algorithm involved doesn’t depend on the type of document we use.

The notebook is divided into the following sections:
1. Importing and preprocessing data
2. Building the model: Multinomial Naive Bayes
3. Building the model: Bernoulli Naive Bayes

### 1. Importing and Preprocessing Data

Let us first look at the sentences and their classes. We have kept the training sentences in file example_train.csv. Test sentences have been put in the file example_test.csv.

In [1]:
import numpy as np
import pandas as pd
import sklearn

# training data
train_docs = pd.read_csv('movie_review_train.csv') 
train_docs

,class,text
0,Pos,a common complaint amongst film critics is ...
1,Pos,whew this film oozes energy the kind of b...
2,Pos,steven spielberg s amistad which is bas...
3,Pos,he has spent his entire life in an awful litt...
4,Pos,being that it is a foreign language film with...
...,...,...
1595,Neg,if anything stigmata should be taken as...
1596,Neg,john boorman s zardoz is a goofy cinemati...
1597,Neg,the kids in the hall are an acquired taste ...
1598,Neg,there was a time when john carpenter was a gr...


So as you can see there are 5 documents (sentences) , 3 are of "education" class and 2 are of "cinema" class.

In [4]:

pos_neg = train_docs['class'].value_counts()
pos_neg

Neg    800
Pos    800
Name: class, dtype: int64

In [6]:
# convert label to a numerical variable
train_docs['class'] = train_docs['class'].map({'Neg':0, 'Pos':1})
train_docs


,class,text
0,1,a common complaint amongst film critics is ...
1,1,whew this film oozes energy the kind of b...
2,1,steven spielberg s amistad which is bas...
3,1,he has spent his entire life in an awful litt...
4,1,being that it is a foreign language film with...
...,...,...
1595,0,if anything stigmata should be taken as...
1596,0,john boorman s zardoz is a goofy cinemati...
1597,0,the kids in the hall are an acquired taste ...
1598,0,there was a time when john carpenter was a gr...


In [40]:
# convert label to a numerical variable
train_docs['text'] = train_docs['text'].str.lstrip()
train_docs


,class,text
0,1,a common complaint amongst film critics is w...
1,1,whew this film oozes energy the kind of br...
2,1,steven spielberg s amistad which is base...
3,1,he has spent his entire life in an awful littl...
4,1,being that it is a foreign language film with ...
...,...,...
1595,0,if anything stigmata should be taken as ...
1596,0,john boorman s zardoz is a goofy cinematic...
1597,0,the kids in the hall are an acquired taste ...
1598,0,there was a time when john carpenter was a gre...


Let's now split the dataframe into X and y labels.

In [48]:
# convert the df to a numpy array 
train_array = train_docs.values

# split X and y
X_train = train_array[:,0]
y_train = train_array[:,1]

# sklearn needs y as integers
#y_train = y_train.astype('int')
y_train = np.asarray(y_train, dtype="|S6")
print("X_train")
print(X_train)
print("y_train")
print(y_train)

X_train
[1 1 1 ... 0 0 0]
y_train
[b'a comm' b'whew  ' b'steven' ... b'the ki' b'there ' b'two pa']


### Creating the Bag of Words Representation

We now have to convert the data into a format which can be used for training the model. We'll use the **bag of words representation** for each sentence (document).

Imagine breaking X in individual words and putting them all in a bag. Then we pick all the unique words from the bag one by one and make a dictionary of unique words. 

This is called **vectorization of words**. We have the class ```CountVectorizer()``` in scikit learn to vectorize the words. 


In [49]:
# create an object of CountVectorizer() class 
from sklearn.feature_extraction.text import CountVectorizer 
# help(CountVectorizer)

In [50]:
vec = CountVectorizer(stop_words='english',min_df=.03, max_df=.8)

Here ```vec``` is an object of class ```CountVectorizer()```. This has a method called  ```fit()``` which converts a corpus of documents to a matrix of 'tokens'.

In [54]:
# fit the vectorizer on training data 
vec.fit(y_train)
vec.vocabulary_

{'comm': 119,
 'whew': 591,
 'steven': 502,
 've': 566,
 'hate': 243,
 'susan': 514,
 'ingred': 262,
 'capsul': 93,
 'bruce': 86,
 'mus': 362,
 'ca': 91,
 'expand': 186,
 'waitin': 574,
 'becaus': 67,
 '197': 6,
 'sha': 473,
 'today': 539,
 'note': 372,
 'bug': 89,
 'near': 366,
 'jerry': 279,
 'ha': 240,
 'da': 136,
 'seen': 466,
 'althou': 37,
 'fee': 195,
 'ta': 520,
 'perhap': 395,
 'kadosh': 289,
 'fi': 199,
 'hum': 256,
 'good': 230,
 'curdle': 135,
 'jarvis': 275,
 'richar': 444,
 'bree': 83,
 'titani': 537,
 'americ': 39,
 '19': 4,
 'wa': 573,
 'ma': 323,
 'arg': 48,
 'felix': 198,
 'know': 296,
 'living': 315,
 'oh': 380,
 'writte': 610,
 'octobe': 379,
 'ok': 381,
 'traili': 546,
 'll': 316,
 'accord': 19,
 'nottin': 375,
 'slaver': 483,
 'reflec': 433,
 'mu': 359,
 'gr': 234,
 'lean': 306,
 '1992': 11,
 'expect': 187,
 'je': 277,
 'aggres': 26,
 'gothic': 233,
 'ess': 180,
 'clue': 117,
 'dr': 166,
 'people': 394,
 'ce': 100,
 'synops': 519,
 'matthe': 334,
 'ro': 446,
 'con

```Countvectorizer()``` has converted the documents into a set of unique words alphabetically sorted and indexed.


**Stop Words**

We can see a few trivial words such as  'and','is','of', etc. These words don't really make any difference in classyfying a document. These are called **stop words**. So we would like to get rid of them. 

We can remove them by passing a parameter stop_words='english' while instantiating ```Countvectorizer()``` as follows: 

In [56]:
# fitting the vectorizer on training data again
# removing the stop words this time
vec = CountVectorizer(stop_words='english')
vec.fit(y_train)
vec.vocabulary_

{'comm': 119,
 'whew': 591,
 'steven': 502,
 've': 566,
 'hate': 243,
 'susan': 514,
 'ingred': 262,
 'capsul': 93,
 'bruce': 86,
 'mus': 362,
 'ca': 91,
 'expand': 186,
 'waitin': 574,
 'becaus': 67,
 '197': 6,
 'sha': 473,
 'today': 539,
 'note': 372,
 'bug': 89,
 'near': 366,
 'jerry': 279,
 'ha': 240,
 'da': 136,
 'seen': 466,
 'althou': 37,
 'fee': 195,
 'ta': 520,
 'perhap': 395,
 'kadosh': 289,
 'fi': 199,
 'hum': 256,
 'good': 230,
 'curdle': 135,
 'jarvis': 275,
 'richar': 444,
 'bree': 83,
 'titani': 537,
 'americ': 39,
 '19': 4,
 'wa': 573,
 'ma': 323,
 'arg': 48,
 'felix': 198,
 'know': 296,
 'living': 315,
 'oh': 380,
 'writte': 610,
 'octobe': 379,
 'ok': 381,
 'traili': 546,
 'll': 316,
 'accord': 19,
 'nottin': 375,
 'slaver': 483,
 'reflec': 433,
 'mu': 359,
 'gr': 234,
 'lean': 306,
 '1992': 11,
 'expect': 187,
 'je': 277,
 'aggres': 26,
 'gothic': 233,
 'ess': 180,
 'clue': 117,
 'dr': 166,
 'people': 394,
 'ce': 100,
 'synops': 519,
 'matthe': 334,
 'ro': 446,
 'con

Notice that the vocabulary has reduced to 12 from 15. Another way of printing the 'vocabulary' is as follows:

In [57]:
# printing feature names
print(vec.get_feature_names())
print(len(vec.get_feature_names()))

['118', '13', '14', '18', '19', '191', '197', '198', '1989', '199', '1990s', '1992', '200', '44', '50', '80', '8mm', 'ab', 'absolu', 'accord', 'actu', 'actual', 'ad', 'adam', 'admi', 'aff', 'aggres', 'ah', 'airpla', 'al', 'albert', 'alchem', 'alcoho', 'alexan', 'alien', 'aliens', 'allen', 'althou', 'ame', 'americ', 'anasta', 'anna', 'anothe', 'apocal', 'apollo', 'appare', 'apr', 'ar', 'arg', 'armage', 'arnold', 'arrivi', 'arye', 'aspiri', 'assume', 'ast', 'att', 'attent', 'august', 'availa', 'bad', 'barb', 'barely', 'based', 'bats', 'battle', 'bea', 'becaus', 'beware', 'bicent', 'big', 'billed', 'billy', 'birthd', 'bl', 'blatan', 'bo', 'bob', 'book', 'books', 'bowfin', 'boy', 'breakd', 'bree', 'bri', 'brian', 'bruce', 'bu', 'buffal', 'bug', 'burnt', 'ca', 'came', 'capsul', 'carla', 'carolc', 'carry', 'cashin', 'castin', 'cather', 'ce', 'certai', 'ch', 'charli', 'chi', 'chill', 'cho', 'chr', 'chris', 'ci', 'cine', 'cinema', 'cir', 'city', 'cl', 'claire', 'clint', 'clue', 'come', 'comm',

So our final dictionary is made of 12 words (after discarding the stop words). Now, to do classification, we need to represent all the documents with these words (or tokens) as features. 

Every document will be converted into a *feature vector* representing presence of these words in that document. Let's convert each of our training documents in to a feature vector.

In [58]:
# another way of representing the features
X_transformed = vec.transform(X_train)
X_transformed

AttributeError: 'int' object has no attribute 'lower'

You can see X_tranformed is a 5 x 12 **sparse matrix**. It has 5 rows for each of our 5 documents and 12 columns each 
for one word of the dictionary which we just created. Let us print X_transformed.

In [59]:
print(X_transformed)

NameError: name 'X_transformed' is not defined

This representation can be understood as follows:

Consider first 4 rows of the output: (0,2), (0,5), (0,7) and (0,11). It says that the first document (index 0) has 
7th , 2nd , 5th and 11th 'word' present in the document, and that they appear only
once in the document- indicated by the right hand column entry. 

Similarly, consider the entry (4,4) (third from bottom). It says that the fifth document has the fifth word present twice. Indeed, the 5th word('good') appears twice in the 5th document. 

In real problems, you often work with large documents and vocabularies, and each document contains only a few words in the vocabulary. So it would be a waste of space to store the vocabulary in a typical dataframe, since most entries would be zero. Also, matrix products, additions etc. are much faster with sparse matrices. That's why we use sparse matrices to store the data.


Let us convert this sparse matrix into a more easily interpretable array:

In [60]:
# converting transformed matrix back to an array
# note the high number of zeros
X_transformed.toarray()

NameError: name 'X_transformed' is not defined

To make the dataset more readable, let us examine the vocabulary and the document-term matrix together in a pandas dataframe. The way to convert a matrix into a dataframe is ```pd.DataFrame(matrix, columns=columns)```.


In [61]:
# converting matrix to dataframe
pd.DataFrame(X_transformed.toarray(), 
             columns=vec.get_feature_names())

NameError: name 'X_transformed' is not defined

This table shows how many times a particular word occurs in document. In other words, this is a frequency table of the words.

A corpus of documents can thus be represented by a matrix with one row per document and one column per
token (e.g. word) occurring in the corpus.

Let's now import and transform the test data as well.

In [62]:
# test data
test_docs = pd.read_csv('example_test.csv') 
test_docs

,Document,Class
0,very good educational institution,education


In [154]:
# convert label to a numerical variable
test_docs['Class'] = test_docs.Class.map({'cinema':0, 'education':1})
test_docs

,Document,Class
0,very good educational institution,1


In [155]:
# convert to numpy array
test_numpy_array = test_docs.values

# split into X and y
X_test = test_numpy_array[:,0]
y_test = test_numpy_array[:,1]

print("X_test")
print(X_test)
print("y_test")
print(y_test)

X_test
['very good educational institution']
y_test
[1]


In [156]:
# transform the test data
# note that you *never* fit on test data, only on training data
# and only transform the test data
X_test_transformed = vec.transform(X_test)
X_test_transformed

<1x12 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [157]:
# convert to non-sparse array
X_test=X_test_transformed.toarray()
X_test

array([[0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]])

Let us summarise all we have done till now:

- ```vect.fit(train)``` learns the vocabulary of the training data
- ```vect.transform(train)``` uses the fitted vocabulary to build a document-term matrix from the training data
- ```vect.transform(test)``` uses the fitted vocabulary to build a document-term matrix from the testing data (and ignores tokens it hasn't seen before)

### 2. Building the Model: Multinomial Naive Bayes

In [158]:
# building a multinomial NB model
from sklearn.naive_bayes import MultinomialNB

# instantiate NB class
mnb=MultinomialNB()

# fitting the model on training data
mnb.fit(X_transformed, y_train)

# note that we are using the sparse matrix X_transformed, 
# though you can also use the non-sparse version
# mnb.fit(X_transformed.toarray(), y_train) 

# predicting probabilities of test data
proba = mnb.predict_proba(X_test)


array([[0.32808399, 0.67191601]])

In [159]:
# probability of each class (test data)
print("probability of test document belonging to class CINEMA" , proba[:,0])
print("probability of test document belonging to class EDUCATION" , proba[:,1])

probability of test document belonging to class CINEMA [0.32808399]
probability of test document belonging to class EDUCATION [0.67191601]


### 3. Building the Model: Bernoulli Naive Bayes

In [160]:
from sklearn.naive_bayes import BernoulliNB

# instantiating bernoulli NB class
bnb=BernoulliNB()

# fitting the model
bnb.fit(X_transformed, y_train)

# also works
# bnb.fit(X_transformed.toarray(), y_train)

# predicting probability of test data
bnb.predict_proba(X_test)
prob_bnb = bnb.predict_proba(X_test)
prob_bnb

array([[0.2326374, 0.7673626]])

In the next notebook, we will use Multinomial and Bernoulli Naive Bayes to solve an interesting real problem - classifying SMSes as spam or ham. We'll also see how to decide the optimal cutoff probability and evaluate the model.
